### Issue 128: Export SAT Trail Geojson

* https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/128
* [this notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue%20128%20Export%20SAT%20Trail%20Geojson.ipynb)
  


In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-07-26 11:09


In [2]:
import requests
import json
from shapely.geometry import LineString, mapping
from geojson import Feature, FeatureCollection

# Step 1: Fetch Wikidata items with P402 (OSM relation)
sparql_query = """
SELECT ?item ?itemLabel ?OSMrel WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825.
  OPTIONAL { ?item wdt:P402 ?OSMrel. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?itemLabel)
"""

def run_sparql(query):
    url = "https://query.wikidata.org/sparql"
    headers = {'Accept': 'application/sparql-results+json'}
    r = requests.get(url, params={'query': query}, headers=headers)
    data = r.json()
    results = []
    for row in data["results"]["bindings"]:
        label = row.get("itemLabel", {}).get("value")
        rel_id = row.get("OSMrel", {}).get("value")
        if rel_id:
            results.append({"label": label, "osm_rel": rel_id})
    return results

# Step 2: Fetch each OSM relation's geometry from Overpass API
def fetch_relation_geometry(osm_rel_id):
    overpass_url = "https://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    relation({osm_rel_id});
    out geom;
    """
    r = requests.post(overpass_url, data={'data': query})
    data = r.json()
    
    features = []
    for element in data.get("elements", []):
        if element["type"] == "relation":
            for member in element.get("members", []):
                if member["type"] == "way" and "geometry" in member:
                    coords = [(pt["lon"], pt["lat"]) for pt in member["geometry"]]
                    features.append(LineString(coords))
    return features

# Step 3: Build GeoJSON
def build_geojson(trails):
    geojson_features = []
    for trail in trails:
        label = trail["label"]
        osm_rel = trail["osm_rel"]
        try:
            geometries = fetch_relation_geometry(osm_rel)
            for geom in geometries:
                feature = Feature(geometry=mapping(geom), properties={"name": label})
                geojson_features.append(feature)
        except Exception as e:
            print(f"Error with relation {osm_rel} ({label}): {e}")
    return FeatureCollection(geojson_features)

# Run everything
if __name__ == "__main__":

    print("Fetching SPARQL trail data...")
    trail_data = run_sparql(sparql_query)
    print(f"Found {len(trail_data)} trails with OSM relations.")

    print("Fetching OSM geometries...")
    geojson = build_geojson(trail_data)

    timestamp = datetime.datetime.now().strftime("%Y%m%d%H_%M_%S")
    output_path = f"trail_sections_{timestamp}.geojson"

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)



Fetching SPARQL trail data...
Found 20 trails with OSM relations.
Fetching OSM geometries...


In [3]:
    end_time = time.time()
    duration = end_time - start_time
    print(f"GeoJSON saved to {output_path}")
    print(f"Finished in {duration:.2f} seconds.")


GeoJSON saved to trail_sections_2025072611_09_31.geojson
Finished in 10.15 seconds.
